## Tokenization lab
LLM's and ChatGPT | Fall 2023 | McSweeney | CUNY Graduate Center

**Due:** October 8


### Background
The purpose of this lab is to explore different tokenization methods. On their own, tokenization methods don't do much. However, they are the starting place for all natural language processing. 


#### Notes
This is a short lab using the same dataset throughout. Feel free to switch it up, but once you are comfortable with how the different alogorithms approach the task of breaking up text, move on. 

You will be using the `datasets` package. You can [install the package](https://pypi.org/project/datasets/) with `$ pip install datasets`. If you do not have `pip` or `conda` installed on your machine, please install it now.

In [ ]:
import nltk
import timeit

from datasets import load_dataset

The next cell is just downloading the dataset. You need to be connected to the internet for this to work. 

This dataset is hosted by [Hugging Face](https://huggingface.co). Hugging Face hosts machine learning models, datasets, and more. We will reference them again. It's a great place to find corpora. 


The dataset is called [American Stories](https://huggingface.co/datasets/dell-research-harvard/AmericanStories). Please skim the Dataset Card. All models and datasets on the Hugging Face hub have these associated cards. 

In [ ]:
# Decide what year you want between 1810 and 1963

my_year = "1960"

# Decide how many articles you want to work with (keep this small - it's slow)
num_articles = 10

#  Download data for your choice of year (1810 to 1963)
dataset = load_dataset("dell-research-harvard/AmericanStories",
    "subset_years",
    year_list=[my_year]
)

# Get the first n articles from that year
# instantiate the counter
i=0
# instantiate the string
my_articles = ''
# loop through each article for that year
for article in dataset[my_year]:
    #the article is a dictionary, 
    #we're getting the text of the article by accessing the key, "article"
    my_articles += article.get('article')
    #add one to our counter
    i+=1
    #if the counter is greater than num_articles-1, stop looping
    if i>(num_articles-1): break
    
#validate that it is what we expect by checking on first 100 characters
print(my_articles[:1000])


This section is for formatting. It removes almost all the markup in these articles. It's a fairly standard set of character encodings.

In [ ]:
#remove new line and other formatting characters
for char in ["\n", "\r", "\d", "\t"]:
    my_articles = my_articles.replace(char, " ")
my_articles[:1000]

# Whitespace tokenization


First we'll just break up the words using whitespace. As noted in class, this is a really common first pass. 

In [ ]:
%%time
#this is a magic function to determine how long a cell takes to run. 
#It MUST be the first thing in a cell

#split the whole string on spaces. This returns a list
whitespace_tokens = my_articles.split(' ')

#check the list
whitespace_tokens[:20]

Note: "µs" is microseconds, or a millionth of a second 1/1,000,000

# Morphological Tokenization 

Lemmatizing is the process of breaking down text into tokens by first breaking it up into "words" and then using syntactic knowledge of the language (in this case, English) to break up the words. 

Princeton maintains the [morphy project](https://wordnet.princeton.edu/documentation/morphy7wn#:~:text=Morphology%20in%20WordNet%20uses%20two,word%20that%20is%20in%20WordNet.), which powers `nltk`'s [WordNet Lemmatizer](https://www.nltk.org/api/nltk.stem.wordnet.html). You do NOT need to read this entire documentation, just acknowledge that it requires a significant amount of knowledge about English in order to make it work. 

In [ ]:
#This lemmatizer is based on the Morphy project above
from nltk.stem import WordNetLemmatizer
 
#Uncomment these two lines - you may need to download these, maybe not. 
#nltk.download('wordnet')
#nltk.download('omw-1.4')
wn_lemmatizer = WordNetLemmatizer()

In [ ]:
%%time

#first we have to split the string on spaces to get "words"
whitespace_tokens = my_articles.split(' ')

my_lemmas = []
for word in whitespace_tokens:
    w = wn_lemmatizer.lemmatize(word)
    my_lemmas.append(w)
my_lemmas[:20]

Notice how much time it takes to tokenize on whitespace versus using morphological rules. Also notice if it produced the output you expected. Sometimes it doesn't. 

ms is a millisecond, or one one thousandth of a second 1/1,000

# Byte Pair Encoding

There are two implementations of BPE here. The first [uses a package (bpe)](https://github.com/soaxelbrooke/python-bpe) that you will have to install using `pip` (see above). 

This will implement the algorithm we covered in class and that you can review at [Hugging Face](https://youtu.be/HEikzVL-lZU).

In [ ]:
from bpe import Encoder


In [ ]:
%%time
whitespace_tokens = my_articles.split(' ')

# calling the Encoder algorithm
# we've specified 100 token vocab and 95% to be tokenized
# the other 5% is transformed into UNK
encoder = Encoder(100, pct_bpe=0.95)
encoder.fit(whitespace_tokens)

In [ ]:
#print(encoder.tokenize(my_articles))

print(next(encoder.inverse_transform(encoder.transform([my_articles]))))